In [1]:
# %pip install tensorflow opencv-python mediapipe scikit-learn matplotlib

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [3]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [5]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                              mp_drawing.DrawingSpec(color=(80,110,10), thickness= 1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80,256,121), thickness= 1, circle_radius=1)
                              )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,22,10), thickness= 2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80,44,121), thickness= 2, circle_radius=2)
                              )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(121,22,76), thickness= 2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121,44,250), thickness= 2, circle_radius=2)
                              )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245,117,66), thickness= 2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245,66,230), thickness= 2, circle_radius=2)
                              )

In [ ]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        #read feed by cap.read()
        ret, frame = cap.read()

        #make the detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)

        #to draw landmarks
        draw_styled_landmarks(image, results)

        #to flip the cam
        flipped_frame = cv2.flip(image, 1) #i have used this to flip the camera

        #to show to screen
        cv2.imshow('OpenCV Feed', flipped_frame) # to undo the flip we can use frame instead of "flipped_frame" in it

        #to break whenever needed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(10)

In [7]:
# draw_styled_landmarks(frame, results)

In [8]:
# plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

In [9]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

Data Path storing

In [10]:
#exported data path
DATA_PATH = os.path.join('MP_Data')
#actions for detection
actions = np.array(['hello','thanks','loveyou'])
#thirty videos worth of data
no_sequences = 30
#30frames in length(videos)
sequence_length = 30

In [11]:
for action in actions:
    for sequence in range(no_sequences): #number sequence
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

Collect Key Points Sequence

In [12]:
# cap = cv2.VideoCapture(0)

# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

#     #NEW #loop through actions
#     for action in actions: #Hello thanks loveyou
#         for sequence in range(no_sequences): # loop 30 times for no sequence
#             for frame_num in range(sequence_length): #loop through each feame i.e 30
                

#                 ret, frame = cap.read()


#                 image, results = mediapipe_detection(frame, holistic)
#                 print(results)

#                 draw_styled_landmarks(image, results)

#                 #NEW #apply wait logic
#                 if frame_num == 0:
#                     cv2.putText(image, 'STARTING COLLECTON',(120,200),
#                                 cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
#                     cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12),
#                                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA) 
#                     cv2.waitKey(2000)
#                 else:
#                     cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12),
#                                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)                    

#                 #NEW #export Keypoints
#                 keypoints =  extract_keypoints(results)
#                 npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
#                 np.save(npy_path, keypoints)

#                 # flipped_frame = cv2.flip(image, 1) #i have used this to flip the camera


#                 cv2.imshow('OpenCV Feed', image) # to undo the flip we can use frame instead of "flipped_frame" in it

#                 #to break whenever needed
#                 if cv2.waitKey(10) & 0xFF == ord('q'):
#                     break
#     cap.release()
#     cv2.destroyAllWindows()
#     cv2.waitKey(10)

Preprocess Data and Create labels

In [13]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [14]:
label_map = {label: num for num ,label in enumerate(actions)}

In [ ]:
label_map

In [16]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])


In [17]:
x = np.array(sequences)

In [ ]:
x.shape

In [19]:
y = to_categorical(labels).astype(int)

In [ ]:
y

In [21]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.05)

In [ ]:
X_train.shape

Build Train LSTM Model

In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
tf.config.set_visible_devices([], 'GPU')  # Disables GPU usage


In [24]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [25]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu',input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [26]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=600, callbacks=[tb_callback])

In [ ]:
model.summary()


In [ ]:
res = model.predict(X_test)

In [ ]:
actions[np.argmax(res[0])]

In [ ]:
actions[y_test[0]]

In [ ]:
model.save('action.h5')

In [33]:
model.load_weights('action.h5')


In [34]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_train)

In [36]:
ytrue = np.argmax(y_train,axis=1).tolist()
yhat = np.argmax(yhat,axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue,yhat)

In [ ]:
accuracy_score(ytrue,yhat)

In [39]:
import cv2

# Define colors
colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245)]

def prob_viz(res, actions, input_frame, colors):
    # Make a copy of the input frame to avoid modifying the original frame
    output_frame = input_frame.copy()
    
    # Loop through the predictions and actions
    for num, prob in enumerate(res):
        # Draw a rectangle with width proportional to the probability
        cv2.rectangle(output_frame, 
                      (0, 60 + num * 40), 
                      (int(prob * 100), 90 + num * 40), 
                      colors[num], -1)
        
        # Add the action name as text next to the rectangle
        cv2.putText(output_frame, 
                    actions[num], 
                    (0, 85 + num * 40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    1, 
                    (255, 255, 255), 2, 
                    cv2.LINE_AA)
        
    return output_frame

In [40]:
# plt.figure(figsize=(18,18))
# plt.imshow(prob_viz(res, actions, image, colors))

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            script = sentence
            if len(sentence) > 5: 
                sentence = sentence[-5:]

        #     # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(10)
    
    with open("script.txt", mode='a') as data_file:
            data_file.write(f"{script}\n")

In [ ]:
model.predict(X_test)